# **Tic-Tac-Toe Experiment**


Run this code after inserting some codes

In [ ]:
#Install qiskit if you have not done already !pip install qiskit --quiet
#Install other packages
!pip install qiskit==0.46.0 --quiet
!pip install pylatexenc --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
from qiskit import *
from qiskit.visualization import plot_histogram
from google.colab import widgets
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, Button, Layout
import ipywidgets
import pandas as pd
import math
import numpy as np
import random
import json
import requests
import qiskit as qk
from IPython.display import clear_output


Now here you will have to write a small function that takes a qiskit ```circuit``` object and runs those instructions on the Quokka.

In [ ]:
def send_to_the_quokka(circuit):
  req_str_qasm = 'http://quokka1.quokkacomputing.com/qsim/qasm' # the URL listening for qasm files


  # create a qasm file from our circuit
  qasmFile = circuit.qasm()


  data = {'script': qasmFile, 'count': 10}
  # create data file to send to Quokka

  result = requests.post(req_str_qasm, json=data)
  json_obj = json.loads(result.content)

  return ''.join(map(str, '')) # convert to a string of binary values

The code for the game comprises the two classes below. This time, try some different moves by allowing the players to add different gates on the circuit.

In [ ]:
class Board:
    def __init__(self):
        self.qc = QuantumCircuit(9, 9)
        self.function = ''
        self.target = -1
        self.tab = []
        self.winsX = 0
        self.winsO = 0

        for idx in range(0, 9):
            self.tab.append({'default':str(idx), 'player':' '})
            self.qc.h(idx)
        self.qc.barrier()

    def make_move(self, cell):
        if self.function == 'H':
            self.qc.h(cell)
            self.tab[int(cell)]['player'] += 'H - '
        elif self.function == 'S':
            self.qc.s(cell)
            self.tab[int(cell)]['player'] += "S - "
        elif self.function == 'T':
            self.qc.t(cell)
            self.tab[int(cell)]['player'] += "T - "
        elif self.function == 'CNOT' and self.target != cell:
            if self.target == cell:
                self.target = -1
            else:
                self.qc.cnot(cell, self.target)
                self.tab[int(cell)]['player'] += "CX - "
                self.tab[int(self.target)]['player'] += "CX - "

    def results(self):
        display(self.qc.draw('mpl'))
        self.qc = QuantumCircuit(9, 9)

    def display(self):
        display(self.qc.draw('mpl'))

    def measure(self):
        self.qc.barrier()
        for i in range(0,9):
            self.qc.measure(i, i)

        output = send_to_the_quokka(self.qc)

        if len(output) >= 9:
          for i in range(0, 9):
              if output[9 - 1 - i] == '0':
                  self.tab[i]['player'] = 'X'
              else:
                  self.tab[i]['player'] = 'O'
          self.winsX = self.countWinners('X')
          self.winsO = self.countWinners('O')
        else:
          print("Error: Measurement output length is less than 9")

    def countWinners(self, player):
        winners = ((0,1,2),(3,4,5),(6,7,8),(0,3,6),(1,4,7),(2,5,8),(0,4,8),(2,4,6))
        wins = 0
        for i in range(len(winners)):
            won = True
            for j in range(len(winners[0])):
                if not self.tab[winners[i][j]]['player'] == player:
                    won = False
            if won:
                wins = wins + 1
        return wins

    def new(self):
        self.tab.clear()
        self.qc = QuantumCircuit(9, 9)
        for idx in range(0,9):
           self.tab.append({'default':str(idx), 'player':''})
           self.qc.h(idx)
        self.qc.barrier()


In [ ]:
class Game:
    def __init__(self):
        self.selecting = False
        self.board = Board()
        self.boardbutton_list = []
        for i in range(0,9):
            button = Button(description=str(i))
            button.on_click(self.handle_game)
            self.boardbutton_list.append(button)

        self.funcbutton_list = []
        self.newButton('Measure')
        self.newButton('H')
        self.newButton('S')
        self.newButton('T')
        self.newButton('CNOT')
        self.printmenu()
        self.printBoard()

    def newButton(self, name):
        function = Button(description=name, layout=Layout(width='86px', height='30px'))
        function.on_click(self.handle_game)
        self.funcbutton_list.append(function)

    def handle_game(self, b):
        try:
            if b.description == 'Measure':
                clear_output()
                self.replay()
                self.board.measure()
                self.scoreboard()
                self.printBoard()
                self.board.results()


            if b.description == 'Replay':
                clear_output()
                self.board.new()
                self.printmenu()
                self.printBoard()

            if int(b.description) >= 0:
                if self.selecting:
                    self.board.target = int(b.description)
                else:
                    clear_output()
                    self.printmenu()
                    self.board.make_move(int(b.description))
                    self.printBoard()
                    self.board.display()
            self.selecting = False
        except ValueError:
            self.board.function = b.description
            self.selecting = False
            if self.board.function == 'CNOT':
                self.selecting = True

    def printmenu(self):
        grid = widgets.Grid(1, 5)
        for (row, col) in grid:
            display(self.funcbutton_list[col])

    def scoreboard(self):
        print("X wins: " + str(self.board.winsX) + "    O wins: " + str(self.board.winsO))

    def replay(self):
        rep = Button(description="Replay")
        rep.on_click(self.handle_game)
        display(rep)

    def printBoard(self):
        grid = widgets.Grid(1, int(np.sqrt(9)), header_row=True, header_column=True)
        for row in range(int(np.sqrt(9))):
              for (useless, col) in grid:
                  print("\n"+self.board.tab[col + row * int(np.sqrt(9))]['player']+"\n")
                  display(self.boardbutton_list[col + row * int(np.sqrt(9))])

## The game! (THESE INSTRUCTIONS SHOULD CHANGE IF YOU CHANGE THE GAME!)
To make a move, click one of the 5 options above the board and then click the location where you want to make that move. To play "SWAP," you need to click two board locations.

Each move adds a gate to the game circuit. They do the following:

* **Measure** ends the round and execute the game circuit on the quantum device. The win conditions will be counted and displayed.
* **H** Adds an $H$ gate at this location.
* **S** Adds an $S$ gate at this location.
* **T** Adds a $T$ gate at this location.
* **CNOT** place a $CNOT$ gate across two location.

As you play, you will see the game circuit and the board. The board shows the sequence of moves, but is not 100% accurate. The game circuit is true state of the game.

In [ ]:
game = Game() #Run this game

,,,,


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Button(description='Measure', layout=Layout(height='30px', width='86px'), style=ButtonStyle())

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Button(description='8', style=ButtonStyle())

<IPython.core.display.Javascript object>